### Unzip data

In [3]:
import zipfile 

with zipfile.ZipFile("D:/Santi/Documents/Jobs projects/Project3/1429_1.csv.zip", "r") as zip_ref:
    zip_ref.extractall('D:/Santi/Documents/Jobs projects/Project3/')

print("Files Uncompressed")

Files Uncompressed


In [77]:
import pandas as pd, numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer #some libraries

In [78]:
csv_data = "1429_1.csv" #or path

df=pd.read_csv(csv_data)
df.head(2)

C:\Users\santi\AppData\Local\Temp\ipykernel_7112\3547152585.py:3: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(csv_data)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman


### Charge and make changes in CSV to key columns

In [2]:
%%writefile ../src/dataio.py
#comment to test overwritting

import pandas as pd, numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer #some libraries
def load_reviews(csv_path):
    df=pd.read_csv(csv_path)
    rename_ft = { "reviews.text": "text",
        "reviews.rating": "rating",
        "name": "product"
    }
    df = df.rename(columns={k: v for k, v in rename_ft.items() if k in df.columns}) #rename for clarifying data
    required = ["text", "product"]
    for col in required:
        if col not in df.columns:
            raise ValueError(f'Miss column required: {col}')
    df = df.dropna(subset=["text", "product"]).reset_index(drop=True) #this reset index for holes after droping NaN
    return df

#Chunking text in pieces with limited characters
def chunk_text(text, max_chars=500):
    text= str(text)
    return [text[i:i+max_chars] for i in range(0, len(text), max_chars)]

#corpus build 
def build_corpus(df: pd.DataFrame, text_col: str = "text", product_col: str = "product", rating_col: str = "rating", max_chars: int = 500) -> pd.DataFrame:
    rows=[]
    for _, r in df.iterrows():
        chunks = chunk_text(r[text_col], max_chars=max_chars)
        for ch in chunks:
            rows.append({
                "product": r[product_col],
                "rating": r.get(rating_col, np.nan), #place nan if raintg_col doesnt exist
                "chunk": ch #create rows for each ch with same product and rating(as it should be) 
            })                       #because it comes from the same text review and product and rating
    return pd.DataFrame(rows)

Overwriting ../src/dataio.py


In [85]:
load_reviews(csv_data)

C:\Users\santi\AppData\Local\Temp\ipykernel_7112\3834594105.py:2: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(csv_data)


,id,product,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,rating,reviews.sourceURLs,text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27894,AVpfl8cLLJeJML43AE3S,New Amazon Kindle Fire Hd 9w Powerfast Adapter...,"B00L9EPT8O,B01E6AO69U",Amazon,"Stereos,Remote Controls,Amazon Echo,Audio Dock...","echowhite/263039693056,echowhite/152558276095,...",Amazon,2016-08-16T00:00:00.000Z,NaN,"2017-09-28T00:00:00Z,2017-09-08T00:00:00Z,2017...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5588528/review...,This is my new favorite device. While not perf...,My new favorite product,NaN,NaN,JefferyP
27895,AVpfl8cLLJeJML43AE3S,New Amazon Kindle Fire Hd 9w Powerfast Adapter...,"B00L9EPT8O,B01E6AO69U",Amazon,"Stereos,Remote Controls,Amazon Echo,Audio Dock...","echowhite/263039693056,echowhite/152558276095,...",Amazon,2016-08-16T00:00:00.000Z,NaN,"2017-09-28T00:00:00Z,2017-09-08T00:00:00Z,2017...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5588528/review...,I got this to basically experiment with. Strai...,Lots of potential!!!,NaN,NaN,shelbydave
27896,AVpfl8cLLJeJML43AE3S,New Amazon Kindle Fire Hd 9w Powerfast Adapter...,"B00L9EPT8O,B01E6AO69U",Amazon,"Stereos,Remote Controls,Amazon Echo,Audio Dock...","echowhite/263039693056,echowhite/152558276095,...",Amazon,2016-08-15T00:00:00.000Z,NaN,"2017-09-28T00:00:00Z,2017-09-08T00:00:00Z,2017...",...,True,NaN,6.0,4.0,http://reviews.bestbuy.com/3545/5588528/review...,Good product that does the basics. Too bad you...,Good First Generation Product

Testing chunk_text

In [81]:
chunk_text("testingchunktext", max_len=7)

['testing', 'chunkte', 'xt']

Testing corpus build

In [82]:
df_test = pd.DataFrame({
    "product": ["Cafetera X", "Audífonos Y", "Mouse Z"],
    "text": [
        "Me encantó la cafetera, prepara el café muy rápido y sabe genial. El depósito es amplio.",
        "Son cómodos pero el sonido es medio; la batería dura poco para llamadas largas.",
        "Buen mouse inalámbrico. El clic es silencioso, aunque el scroll podría ser más suave."
    ],
    "rating": [5, 3, np.nan]  # uno con NaN para probar el default
})
df_test.head()


,product,text,rating
0,Cafetera X,"Me encantó la cafetera, prepara el café muy rá...",5.0
1,Audífonos Y,Son cómodos pero el sonido es medio; la baterí...,3.0
2,Mouse Z,"Buen mouse inalámbrico. El clic es silencioso,...",NaN


In [83]:
corpus = build_corpus(df_test, "text", "product", "rating", 45)
corpus.head(10)

,product,rating,chunk
0,Cafetera X,5.0,"Me encantó la cafetera, prepara el café muy r"
1,Cafetera X,5.0,ápido y sabe genial. El depósito es amplio.
2,Audífonos Y,3.0,Son cómodos pero el sonido es medio; la bater
3,Audífonos Y,3.0,ía dura poco para llamadas largas.
4,Mouse Z,NaN,Buen mouse inalámbrico. El clic es silencioso
5,Mouse Z,NaN,", aunque el scroll podría ser más suave."
